## 2. Morphological Image Processing

### Util functions

In [9]:
from skimage import color, morphology ,io
from scipy import ndimage as ndi
from matplotlib import patches
import numpy as np
import matplotlib.pyplot as plt

def filter_demo(image, operation, vmax=1):
    iter_kernel_and_subimage = iter_kernel(image)
    structure = morphology.disk(1)
    dilation = morphology.binary_dilation(image, structure)
    erosion = morphology.binary_erosion(image, structure)
    
    image_cache = []

    def filter_step(i_step):
        while i_step >= len(image_cache):
            filtered = np.zeros(shape=image.shape) if i_step == 0 else image_cache[-1][-1][-1]
            filtered = filtered.copy()

            (i, j), mask = next(iter_kernel_and_subimage)
            filter_overlay = color.label2rgb(mask, image, bg_label=0,
                                             colors=('red', 'yellow'))
            if operation == 'dilation':
                filtered[i, j] = dilation[i, j]
            elif operation == 'erosion':
                filtered[i, j] = erosion[i, j]             

            count = np.sum(mask.astype(np.bool_) & image.astype(np.bool_))
            if count == np.sum(mask.astype(np.bool_)): # fit
                state = 1
            elif count == 0: # miss
                state = 3
            else: # hit
                state = 2
                
            image_cache.append(((i, j), (state), (filter_overlay, filtered)))

        (i, j), state, images = image_cache[i_step]
        fig, axes = plt.subplots(1, len(images), figsize=(15, 10))
        
        if state == 1:
            plt.title('Fit!!', fontsize=25)
        elif state == 2:
            plt.title('Hit!!', fontsize=25)
        else:
            plt.title('Miss!!', fontsize=25)
        
        for ax, imc in zip(axes, images):
            ax.imshow(imc, vmax=vmax)
            rect = patches.Rectangle([j - 0.5, i - 0.5], 1, 1, color='yellow', fill=False)
            ax.add_patch(rect)

        plt.show()
    return filter_step


def filter_interactive_demo(image, operation):
    from ipywidgets import IntSlider, interact, Layout
    filter_step = filter_demo(image, operation)
    step_slider = IntSlider(min=0, max=image.size-1, value=0, layout=Layout(width='50%') )
    interact(filter_step, i_step=step_slider)


def iter_kernel(image, size=1):

    width = 2*size + 1
    for (i, j), pixel in iter_pixels(image):
        mask = np.zeros(image.shape, dtype='int16')
        mask[i, j] = 1
        mask = ndi.grey_dilation(mask, size=width, structure=morphology.disk(1))-1
        mask[i, j] = 2
        subimage = image[bounded_slice((i, j), image.shape[:2], size=size)]
        yield (i, j), mask


def iter_pixels(image):
    """ Yield pixel position (row, column) and pixel intensity. """
    height, width = image.shape[:2]
    for i in range(height):
        for j in range(width):
            yield (i, j), image[i, j]


def bounded_slice(center, xy_max, size=1, i_min=0):
    slices = []
    for i, i_max in zip(center, xy_max):
        slices.append(slice(max(i - size, i_min), min(i + size + 1, i_max)))
    return tuple(slices)

### Erosion & Dilation

In [12]:
example = io.imread('binary.png', as_gray=True) > 0.5

filter_interactive_demo(example, 'erosion')

interactive(children=(IntSlider(value=0, description='i_step', layout=Layout(width='50%'), max=99), Output()),…

In [11]:
filter_interactive_demo(example, 'dilation')

interactive(children=(IntSlider(value=0, description='i_step', layout=Layout(width='50%'), max=99), Output()),…